<h1> <center>
🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐁🐁🐁🐁🐁🐁🐁🐁🐁🐁<br>
🐀 MABe Classical Classification: Baseline 💪 🐁<br>
🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐀🐁🐁🐁🐁🐁🐁🐁🐁🐁🐁
</center>
</h1>

<p align="center"><img src="https://images.aicrowd.com/uploads/ckeditor/pictures/324/content_task1_structure.png">
</p>

# Approach used in Notebook

* Add future and past frames to make an input window.
* Normalize and flatten the inputs
* Use 1D convolutional network with time as the convolution axis
* Supervised classification on each input window



# Import necessary modules and packages 📚


In [1]:
import numpy as np
import os

from tensorflow import keras
import tensorflow as tf
from keras.models import Sequential
import keras.layers as layers
import tensorflow_addons as tfa

import sklearn
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy
import tqdm
import gc

import itertools

import h2o
from h2o.automl import H2OAutoML

from collections import defaultdict

In [182]:
import autosklearn.classification
from sklearn.model_selection import GroupShuffleSplit

# Download the dataset 📲

Please get your API key from https://www.aicrowd.com/participants/me


In [2]:
API_KEY = "0ba231d61506b40a4ae00df011cf0cb9"
!aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


# Load Data
The dataset files are python dictionaries, [this](https://colab.research.google.com/drive/1ddCX-TAdEcsUaGf09f5Glgr_G57FMK_O#scrollTo=JPsfxdl2GMcM&line=18&uniqifier=1) is a descirption of how the data is organized.



In [3]:
train = np.load('data/train.npy',allow_pickle=True).item()
test = np.load('data/test.npy',allow_pickle=True).item()
sample_submission = np.load('data/sample_submission.npy',allow_pickle=True).item()

In [ ]:
def seed_everything(seed):
  np.random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  tf.random.set_seed(seed)

seed=2021
seed_everything(seed)

## Dataset Specifications 💾

-   **`train.npy`** - Training set for the task, which follows the following schema:

<p align="left" style="padding: 30px">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img width="600px" src="https://i.imgur.com/GddvoT2.png">
</p>


-   **`test-release.npy`** - Test set for the task, which follows the following schema :

<p align="left" style="padding: 30px">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<img width="600px" src="https://i.imgur.com/vfa8Y4V.png">
</p>

-   **`sample_submission.npy`** - Template for a sample submission which follows the following schema

```json
{
    "<sequence_id-1>" : [0, 0, 1, 2, ...],
    "<sequence_id-2>" : [0, 1, 2, 0, ...]
}
```

Each key in the dictionary here refers to the unique sequence id obtained for the sequences in the test set.
The value for each of the keys is expected to hold a list of corresponing annotations. The annotations are represented by the index of the corresponding annotation words in the vocabular provided in the test set.

# Training The Model 🏋️‍♂️

The given MABe dataset contain many sequences of time series data, each frame has its own behavior label. Training on just a single frame does not give good results due to less information. 

So here past and future frames are also added to each input. But also all the frames are not concatenated as as the boundaries of the past and future frames need to stay separate for each video.


In [173]:
mouse_ids = ['mouse_0', 'mouse_1']
xy_ids = ['x', 'y']
bodypart_ids = ['nose', 'l_ear', 'r_ear', 'neck', 'l_hip', 'r_hip', 'tail_base']
colnames = ['_'.join(a) for a in itertools.product(mouse_ids, xy_ids, bodypart_ids)]

In [172]:
def make_df(raw_data, add_annotations = True):
    df = []
    labels = []
    seqs = []
    for seq_id in raw_data['sequences']:
        pts = raw_data['sequences'][seq_id]['keypoints']
        for idx in range(len(pts)):
            data = pts[idx].flatten()
            df.append(list(data))
        if 'annotations' in raw_data['sequences'][seq_id] and add_annotations:
            labels += list(train['sequences'][seq_id]['annotations'])
        seqs += [seq_id]*len(pts)

    #Make this a dataframe
    df_ = pd.DataFrame(df, columns = colnames)
    if len(labels) > 0 and add_annotations:
        df_['annotation'] = labels
    df_['seq_id'] = seqs
    return df_

In [174]:
test_df = make_df(test)
train_df.to_csv('./data/test_df.csv')

In [89]:
train_df = make_df(train)
train_df.to_csv('./data/train_df.csv')

In [194]:
#Feature engineering:
# Difference between each body part
def make_features(df):

    features_df = df.copy()

    for xy in xy_ids:
        for i, bp1 in enumerate(bodypart_ids):
            for j, bp2 in enumerate(bodypart_ids):
                if i < j:
                    for mouse_id in mouse_ids:
                        #We can compute the intra-mouse difference
                        f1 = '_'.join([mouse_id, xy, bp1])
                        f2 = '_'.join([mouse_id, xy, bp2])
                        f_new = '_'.join([mouse_id, xy, bp1, bp2])
                        features_df[f_new] = features_df[f1] - features_df[f2]
                #Inter-mouse difference
                f1 = '_'.join([mouse_ids[0], xy, bp1])
                f2 = '_'.join([mouse_ids[1], xy, bp2])
                f_new = '_'.join(['M0_M1', xy, bp1, bp2])
                features_df[f_new] = features_df[f1] - features_df[f2]
    #Remove base features
    features_df = features_df.drop(columns = colnames)
    
    hashmap = {k: i for (i,k) in enumerate(list(set(features_df['seq_id'])))}
    reversemap = {i: k for (i,k) in enumerate(list(set(features_df['seq_id'])))}
    features_df['seq_id_'] = [hashmap[i] for i in features_df['seq_id']]
    features_df['seq_id'] = features_df['seq_id_']
    features_df.drop(columns = 'seq_id_')
    return features_df, reversemap

In [195]:
train_features, train_map = make_features(train_df)
test_features, test_map = make_features(test_df)

In [99]:
train_features.to_csv('./data/train_features_differences.csv')

In [181]:
test_features.to_csv('./data/test_features_differences.csv')

## H2o AutoML

In [119]:
h2o.init(nthreads = 30, max_mem_size = '120G', min_mem_size = '60G', log_dir = './logs/')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.20.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.20.04, mixed mode, sharing)
  Starting server from /home/blansdel/anaconda3/envs/ml/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyndl8u3t
  JVM stdout: /tmp/tmpyndl8u3t/h2o_blansdel_started_from_python.out
  JVM stderr: /tmp/tmpyndl8u3t/h2o_blansdel_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 10 days
H2O_cluster_name:,H2O_from_python_blansdel_3ig0qj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,120 Gb
H2O_cluster_total_cores:,128
H2O_cluster_allowed_cores:,30
H2O_cluster_status:,"accepting new members, healthy"


In [120]:
data_path = "./data/train_features_differences.csv"# Load data into H2O
df = h2o.import_file(data_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [121]:
y = 'annotation'
splits = 'seq_id'
x = df.columns
x.remove(y)
x.remove(splits)
x.remove('C1')

#aml = H2OAutoML(max_runtime_secs=360, seed=1)
#aml.train(x=x, y=y, training_frame=df, fold_column=splits)

In [122]:
#len(df)
df['seq_id'] = (df['seq_id']/10).ceil()

In [123]:
df

C1,annotation,seq_id,M0_M1_x_nose_nose,mouse_0_x_nose_l_ear,mouse_1_x_nose_l_ear,M0_M1_x_nose_l_ear,mouse_0_x_nose_r_ear,mouse_1_x_nose_r_ear,M0_M1_x_nose_r_ear,mouse_0_x_nose_neck,mouse_1_x_nose_neck,M0_M1_x_nose_neck,mouse_0_x_nose_l_hip,mouse_1_x_nose_l_hip,M0_M1_x_nose_l_hip,mouse_0_x_nose_r_hip,mouse_1_x_nose_r_hip,M0_M1_x_nose_r_hip,mouse_0_x_nose_tail_base,mouse_1_x_nose_tail_base,M0_M1_x_nose_tail_base,M0_M1_x_l_ear_nose,M0_M1_x_l_ear_l_ear,mouse_0_x_l_ear_r_ear,mouse_1_x_l_ear_r_ear,M0_M1_x_l_ear_r_ear,mouse_0_x_l_ear_neck,mouse_1_x_l_ear_neck,M0_M1_x_l_ear_neck,mouse_0_x_l_ear_l_hip,mouse_1_x_l_ear_l_hip,M0_M1_x_l_ear_l_hip,mouse_0_x_l_ear_r_hip,mouse_1_x_l_ear_r_hip,M0_M1_x_l_ear_r_hip,mouse_0_x_l_ear_tail_base,mouse_1_x_l_ear_tail_base,M0_M1_x_l_ear_tail_base,M0_M1_x_r_ear_nose,M0_M1_x_r_ear_l_ear,M0_M1_x_r_ear_r_ear,mouse_0_x_r_ear_neck,mouse_1_x_r_ear_neck,M0_M1_x_r_ear_neck,mouse_0_x_r_ear_l_hip,mouse_1_x_r_ear_l_hip,M0_M1_x_r_ear_l_hip,mouse_0_x_r_ear_r_hip,mouse_1_x_r_ear_r_hip,M0_M1_x_r_ear_r_hip,mouse_0_x_r_ear_tail_base,mouse_1_x_r_ear_tail_base,M0_M1_x_r_ear_tail_base,M0_M1_x_neck_nose,M0_M1_x_neck_l_ear,M0_M1_x_neck_r_ear,M0_M1_x_neck_neck,mouse_0_x_neck_l_hip,mouse_1_x_neck_l_hip,M0_M1_x_neck_l_hip,mouse_0_x_neck_r_hip,mouse_1_x_neck_r_hip,M0_M1_x_neck_r_hip,mouse_0_x_neck_tail_base,mouse_1_x_neck_tail_base,M0_M1_x_neck_tail_base,M0_M1_x_l_hip_nose,M0_M1_x_l_hip_l_ear,M0_M1_x_l_hip_r_ear,M0_M1_x_l_hip_neck,M0_M1_x_l_hip_l_hip,mouse_0_x_l_hip_r_hip,mouse_1_x_l_hip_r_hip,M0_M1_x_l_hip_r_hip,mouse_0_x_l_hip_tail_base,mouse_1_x_l_hip_tail_base,M0_M1_x_l_hip_tail_base,M0_M1_x_r_hip_nose,M0_M1_x_r_hip_l_ear,M0_M1_x_r_hip_r_ear,M0_M1_x_r_hip_neck,M0_M1_x_r_hip_l_hip,M0_M1_x_r_hip_r_hip,mouse_0_x_r_hip_tail_base,mouse_1_x_r_hip_tail_base,M0_M1_x_r_hip_tail_base,M0_M1_x_tail_base_nose,M0_M1_x_tail_base_l_ear,M0_M1_x_tail_base_r_ear,M0_M1_x_tail_base_neck,M0_M1_x_tail_base_l_hip,M0_M1_x_tail_base_r_hip,M0_M1_x_tail_base_tail_base,M0_M1_y_nose_nose,mouse_0_y_nose_l_ear,mouse_1_y_nose_l_ear,M0_M1_y_nose_l_ear,mouse_0_y_nose_r_ear,mouse_1_y_nose_r_ear,M0_M1_y_nose_r_ear,mouse_0_y_nose_neck,mouse_1_y_nose_neck,M0_M1_y_nose_neck,mouse_0_y_nose_l_hip,mouse_1_y_nose_l_hip,M0_M1_y_nose_l_hip,mouse_0_y_nose_r_hip,mouse_1_y_nose_r_hip,M0_M1_y_nose_r_hip,mouse_0_y_nose_tail_base,mouse_1_y_nose_tail_base,M0_M1_y_nose_tail_base,M0_M1_y_l_ear_nose,M0_M1_y_l_ear_l_ear,mouse_0_y_l_ear_r_ear,mouse_1_y_l_ear_r_ear,M0_M1_y_l_ear_r_ear,mouse_0_y_l_ear_neck,mouse_1_y_l_ear_neck,M0_M1_y_l_ear_neck,mouse_0_y_l_ear_l_hip,mouse_1_y_l_ear_l_hip,M0_M1_y_l_ear_l_hip,mouse_0_y_l_ear_r_hip,mouse_1_y_l_ear_r_hip,M0_M1_y_l_ear_r_hip,mouse_0_y_l_ear_tail_base,mouse_1_y_l_ear_tail_base,M0_M1_y_l_ear_tail_base,M0_M1_y_r_ear_nose,M0_M1_y_r_ear_l_ear,M0_M1_y_r_ear_r_ear,mouse_0_y_r_ear_neck,mouse_1_y_r_ear_neck,M0_M1_y_r_ear_neck,mouse_0_y_r_ear_l_hip,mouse_1_y_r_ear_l_hip,M0_M1_y_r_ear_l_hip,mouse_0_y_r_ear_r_hip,mouse_1_y_r_ear_r_hip,M0_M1_y_r_ear_r_hip,mouse_0_y_r_ear_tail_base,mouse_1_y_r_ear_tail_base,M0_M1_y_r_ear_tail_base,M0_M1_y_neck_nose,M0_M1_y_neck_l_ear,M0_M1_y_neck_r_ear,M0_M1_y_neck_neck,mouse_0_y_neck_l_hip,mouse_1_y_neck_l_hip,M0_M1_y_neck_l_hip,mouse_0_y_neck_r_hip,mouse_1_y_neck_r_hip,M0_M1_y_neck_r_hip,mouse_0_y_neck_tail_base,mouse_1_y_neck_tail_base,M0_M1_y_neck_tail_base,M0_M1_y_l_hip_nose,M0_M1_y_l_hip_l_ear,M0_M1_y_l_hip_r_ear,M0_M1_y_l_hip_neck,M0_M1_y_l_hip_l_hip,mouse_0_y_l_hip_r_hip,mouse_1_y_l_hip_r_hip,M0_M1_y_l_hip_r_hip,mouse_0_y_l_hip_tail_base,mouse_1_y_l_hip_tail_base,M0_M1_y_l_hip_tail_base,M0_M1_y_r_hip_nose,M0_M1_y_r_hip_l_ear,M0_M1_y_r_hip_r_ear,M0_M1_y_r_hip_neck,M0_M1_y_r_hip_l_hip,M0_M1_y_r_hip_r_hip,mouse_0_y_r_hip_tail_base,mouse_1_y_r_hip_tail_base,M0_M1_y_r_hip_tail_base,M0_M1_y_tail_base_nose,M0_M1_y_tail_base_l_ear,M0_M1_y_tail_base_r_ear,M0_M1_y_tail_base_neck,M0_M1_y_tail_base_l_hip,M0_M1_y_tail_base_r_hip,M0_M1_y_tail_base_tail_base,seq_id_
0,3,3,107.304,32,-35,72.3043,50,24,131.304,55,-9,98.3043,11

In [124]:
aml = H2OAutoML(max_runtime_secs=1200)
aml.train(x=x, y=y, training_frame=df, fold_column=splits)

AutoML progress: |
17:46:39.270: Fold column seq_id will be used for cross-validation. nfolds parameter will be ignored.

█Failed polling AutoML progress log: Unexpected HTTP error: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Failed polling AutoML progress log: Local server has died unexpectedly. RIP.


H2OConnectionError: Local server has died unexpectedly. RIP.

In [82]:
lb = aml.leaderboard
lb.head()

This H2OFrame is empty.


In [76]:
h2o.cluster().shutdown()

H2O session _sid_a4a7 closed.


### Use autosklearn instead

In [130]:
X_train = train_features.drop(columns = ['annotation', 'seq_id'])
y_train = train_features['annotation']
groups = train_features['seq_id']

In [185]:
X_test = test_features.drop(columns = ['seq_id'])
groups_test = test_features['seq_id']

In [164]:
cls = autosklearn.classification.AutoSklearnClassifier(resampling_strategy = GroupShuffleSplit,
                                                       resampling_strategy_arguments = {'n_splits':5, 'test_size':7,
                                                                                       'groups': groups},
                                                      memory_limit = 100000,
                                                      n_jobs = 1, tmp_folder = './tmp', 
                                                      time_)

In [165]:
cls.fit(X_train, y_train)

AutoSklearnClassifier(memory_limit=100000, n_jobs=1, per_run_time_limit=360,
                      resampling_strategy=<class 'sklearn.model_selection._split.GroupShuffleSplit'>,
                      resampling_strategy_arguments={'groups': 0         29
1         29
2         29
3         29
4         29
          ..
507733    54
507734    54
507735    54
507736    54
507737    54
Name: seq_id, Length: 507738, dtype: int64,
                                                     'n_splits': 5,
                                                     'test_size': 7},
                      tmp_folder='./tmp')

In [17]:
def normalize_data(orig_pose_dictionary):
  for key in orig_pose_dictionary:
    X = orig_pose_dictionary[key]['keypoints']
    X = X.transpose((0,1,3,2)) #last axis is x, y coordinates
    X[..., 0] = X[..., 0]/1024
    X[..., 1] = X[..., 1]/570
    orig_pose_dictionary[key]['keypoints'] = X
  return orig_pose_dictionary

In [171]:
cls.sprint_statistics()

'auto-sklearn results:\n  Dataset name: c2b80cda-82eb-11eb-9cda-bbc369772279\n  Metric: accuracy\n  Best validation score: 0.851898\n  Number of target algorithm runs: 14\n  Number of successful target algorithm runs: 2\n  Number of crashed target algorithm runs: 0\n  Number of target algorithms that exceeded the time limit: 10\n  Number of target algorithms that exceeded the memory limit: 2\n'

In [187]:
cls.refit(X_train.copy(), y_train.copy())

AutoSklearnClassifier(memory_limit=100000, n_jobs=1, per_run_time_limit=1800,
                      resampling_strategy=<class 'sklearn.model_selection._split.GroupShuffleSplit'>,
                      resampling_strategy_arguments={'groups': 0         29
1         29
2         29
3         29
4         29
          ..
507733    54
507734    54
507735    54
507736    54
507737    54
Name: seq_id, Length: 507738, dtype: int64,
                                                     'n_splits': 5,
                                                     'test_size': 7},
                      time_left_for_this_task=18000, tmp_folder='./tmp')

In [188]:
#Make the final prediction on the test data
predictions = cls.predict(X_test)

In [196]:
#Map indices back to group names
test_map

{0: 'd8a38e06e6',
 1: '99a26aeec6',
 2: '025c8b569d',
 3: '1cfeb392db',
 4: '9b4f56fbf8',
 5: 'ab22c7acaf',
 6: 'a2eb262508',
 7: '322890acca',
 8: '84cbf9e1ec',
 9: '7c2beeda6f',
 10: 'd069aac797',
 11: 'a43a63fad0',
 12: '4773f200ba',
 13: 'f61a0cce95',
 14: 'b48237e671',
 15: '6499b4ecb3',
 16: 'd0a406a1c8',
 17: 'b9279ee3a6',
 18: 'ddb2a92cad',
 19: '2079eb6797',
 20: 'b3c5e8af19',
 21: 'ef7579bdea',
 22: 'c29e04feb3',
 23: '3490660099',
 24: '7cc2b2a9af',
 25: '8f96457fd1',
 26: '06ef843cac',
 27: '7ba52bdd90',
 28: 'dde3bb1838',
 29: '3879958005',
 30: '48089d4b1e',
 31: '75d44f92af',
 32: 'd842416f1e',
 33: '5220d8e090',
 34: '563934d439',
 35: 'ffb8445762',
 36: 'e72393959f',
 37: 'cf7aecd8af',
 38: '1d00499404',
 39: 'f53dc77eb3',
 40: '3565c9e2f0',
 41: 'b48828025a',
 42: '3f2be65ff4',
 43: '25637cfa76',
 44: 'bd08cc7d33',
 45: 'a8a3392426',
 46: '0d90631f71',
 47: '221f936a17',
 48: 'a564d43bf6',
 49: 'b65563c41c',
 50: '1bfd82f9e4',
 51: '5bef03e8eb',
 52: '6f46b4c8d3',
 53

In [201]:
submission = defaultdict(list)

In [202]:
#Make the results dict
for idx in range(len(predictions)):
    submission[test_map[groups_test[idx]]].append(predictions[idx])

In [190]:
cls.show_models()

"[(0.700000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'gradient_boosting', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'median', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'feature_agglomeration', 'classifier:gradient_boosting:early_stop': 'valid', 'classifier:gradient_boosting:l2_regularization': 9.674948183980905e-09, 'classifier:gradient_boosting:learning_rate': 0.014247987845444413, 'classifier:gradient_boosting:loss': 'auto', 'classifier:gradient_boosting:max_bins': 255, 'classifier:gradient_boosting:max_depth': 'None', 'classifier:gradient_boosting:max_leaf_nodes': 55, 'classifier:gradient_boosting:min_samples_leaf': 164, 'classifier:gradient_boostin

In [197]:
groups_test

0          438
1          438
2          438
3          438
4          438
          ... 
8168486    323
8168487    323
8168488    323
8168489    323
8168490    323
Name: seq_id, Length: 8168491, dtype: int64

In [ ]:
#Measure performance

#How to do this without leaking training data into validation?

#

# Validate the submission ✅
The submssion should follow these constraints:

1.   It should be a dictionary
2.   It should be have same keys as sample_submission
3.   The lengths of the arrays are same
4.   All values are intergers

You can use the helper function below to check these



In [203]:
def validate_submission(submission, sample_submission):
    if not isinstance(submission, dict):
      print("Submission should be dict")
      return False

    if not submission.keys() == sample_submission.keys():
      print("Submission keys don't match")
      return False
    
    for key in submission:
      sv = submission[key]
      ssv = sample_submission[key]
      if not len(sv) == len(ssv):
        print(f"Submission lengths of {key} doesn't match")
        return False
    
    for key, sv in submission.items():
      if not all(isinstance(x, (np.int32, np.int64, int)) for x in list(sv)):
        print(f"Submission of {key} is not all integers")
        return False
    
    print("All tests passed")
    return True

In [204]:
validate_submission(submission, sample_submission)

All tests passed


True

## Save the prediction as `npy` 📨

In [205]:
np.save("submission.npy", submission)

## Submit to AIcrowd 🚀

In [206]:
!aicrowd submission create -c mabe-task-1-classical-classification -f submission.npy

submission.npyon.npy ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • ? • -:--:-- ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • ? • -:--:-- ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • ? • -:--:-- ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • ? • -:--:-- ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • ? • -:--:-- ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • ? • -:--:-- ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • 1.5 MB/s • 0:02:45 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • 1.7 MB/s • 0:02:33 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • 2.1 MB/s • 0:02:03 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • 2.3 MB/s • 0:01:49 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.0/253.3 MB • 2.5 MB/s • 0:01:40 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.1/253.3 MB • 2.7 MB/s • 0:01:34 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.1/253.3 MB • 2.8 MB/s • 0:01:29 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.1/253.3 MB • 3.0 MB/s • 0:01:26 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.0% • 0.1/253.3 MB • 3.0 MB/s

submission.npy ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 0.9/253.3 MB • 1.6 MB/s • 0:02:39 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:39 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:39 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:38 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:37 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:37 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:36 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:35 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:34 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.6 MB/s • 0:02:34 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.7 MB/s • 0:02:33 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.7 MB/s • 0:02:32 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.7 MB/s • 0:02:31 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.0/253.3 MB • 1.7 MB/s • 0:02:31 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.4% • 1.1/253.3 MB • 1.7 MB/s • 0:0

submission.npy ━━━━━━━━━━━━━━━━━━━━━━━━ 0.8% • 1.9/253.3 MB • 2.2 MB/s • 0:01:54 ━━━━━━━━━━━━━━━━━━━━━━━━ 0.8% • 1.9/253.3 MB • 2.0 MB/s • 0:02:07

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



submission.npy ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.8/253.3 MB • 2.0 MB/s • 0:02:05 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.8/253.3 MB • 2.0 MB/s • 0:02:05 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.8/253.3 MB • 2.0 MB/s • 0:02:05 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.8/253.3 MB • 2.0 MB/s • 0:02:05 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.8/253.3 MB • 2.0 MB/s • 0:02:05 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.8/253.3 MB • 2.0 MB/s • 0:02:05 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.8/253.3 MB • 2.0 MB/s • 0:02:05 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.9/253.3 MB • 2.0 MB/s • 0:02:05 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.9/253.3 MB • 2.0 MB/s • 0:02:04 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.9/253.3 MB • 2.0 MB/s • 0:02:04 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.9/253.3 MB • 2.0 MB/s • 0:02:04 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.9/253.3 MB • 2.0 MB/s • 0:02:04 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.9/253.3 MB • 2.0 MB/s • 0:02:04 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.9/253.3 MB • 2.0 MB/s • 0:02:04 ━━━━━━━━━━━━━━━━━━━━━━━━ 1.5% • 3.9/253.3 MB • 2.0 MB/s • 0:0

...

submission.npy ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.1/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.2/253.3 MB • 1.5 MB/s • 0:00:04 ━━━━━━━━━━━━━━━━━━━━╸ 98.0% • 248.3/253.3 MB • 1.5 MB/s • 0:0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



submission.npy ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.8/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.8/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.8/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.0% • 250.8/253.3 MB • 1.5 MB/s • 0:0

submission.npy ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.7/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.8/253.3 MB • 1.5 MB/s • 0:00:02 ━━━━━━━━━━━━━━━━━━━━╸ 99.4% • 251.8/253.3 MB • 1.5 MB/s • 0:0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

